In [ ]:
import pandas as pd
from stage import fullPrint, getIndustryStage
from datetime import datetime, timedelta
import numpy as np
import pickle
import os
import investpy
import swifter

PEAK = 12
allList = []
industryList = pd.read_pickle("stockData/industryList.pkl")

SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}

i = 0
now = datetime.now()

In [ ]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
WMA30SLOPE = 7
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,WMA30SLOPE]/df.iat[index-1,WMA30SLOPE]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
        
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])

In [ ]:
spDf = investpy.get_index_historical_data(index='S&P 500', country='United States', from_date='01/01/2000', to_date='01/01/2020')
spDf = spDf[~spDf.index.duplicated()]
spDf['Close'] = spDf['Close'].rolling(window=5).apply(lambda x: np.average(x))
spDf['Dates'] = pd.to_datetime(spDf.index)
spDf = spDf[spDf['Dates'].dt.weekday == 4]
spDf = spDf.drop('Dates', axis = 1)
spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf['WMA30'] = spDf['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
spDf['WMA30Slope'] = spDf.swifter.apply(lambda x: calculateSlopeOfTicker(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf.to_pickle("stockData/Spy.pkl")

In [ ]:
for industry in industryList:
    print(industry)
    try:
        stockDf = pd.read_pickle('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl')
        if stockDf['Close'].isnull().values.any():
            os.remove('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl')
        stockDf['fiveYearHigh'] = stockDf['Close'].rolling(window=260).max().shift(1).fillna(0)
        stockDf['WMA30'] = stockDf['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['Volume'] = stockDf['Volume'].replace(to_replace=0, method='ffill')
        stockDf['volumePerc'] = stockDf['Volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        stockDf.to_pickle('stockData/industriesData/'+str(industry[0])+'/'+str(industry[1])+'.pkl')
    except:
        pass

In [ ]:
transactionTemplate = investpy.get_index_historical_data(index='DJ Commodity Chemicals',
                                        country='United States',
                                        from_date='01/01/2010',
                                        to_date='01/01/2020').drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate = transactionTemplate[~transactionTemplate.index.duplicated()]
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [ ]:
for ticker in nasdaqList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

In [ ]:
now = datetime.now()
monday = now - timedelta(days = now.weekday())
now = now.strftime('%Y-%m-%d')
df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
df['Dates'] = pd.to_datetime(df.index)
df = df[df['Dates'].dt.weekday == 4]
df = df.drop('Dates', axis = 1)
df

In [ ]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate['Dates'] = pd.to_datetime(transactionTemplate.index)
transactionTemplate = transactionTemplate[transactionTemplate['Dates'].dt.weekday == 4]
transactionTemplate = transactionTemplate.drop('Dates', axis = 1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [ ]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            if  df.at[date,'MA30Slope'] > 0.992549883242513 and df.loc[date,'RS'] > -0.3044455098712273:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1


In [ ]:
removeList = []
CLOSE = 0
TICKER = 2
for ticker in nasdaqList:
    try:
        df = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
        lastClose = df.iat[0,CLOSE]
        isFirst = True
        index = 0
        for date in df.Stage:
            if isFirst == True:
                isFirst = False
                continue
            if (df.iat[index,CLOSE] >= lastClose * 5):
                removeList.append(ticker)
                break
            lastClose = df.iat[index,CLOSE]
            index += 1
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        buyDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        if stockDf['close'].isnull().values.any() or buyDf['open'].isnull().values.any():
            print(ticker)
            removeList.append(ticker)
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            os.remove('stockData/nyseNasdaq/'+ticker+'Buy.pkl')
        df = getStage(ticker,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
        if df.empty:
            removeList.append(ticker)
    except:
        removeList.append(ticker)
print(removeList)

In [ ]:
for ticker in removeList:
    nasdaqList.remove(ticker)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)